In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys

import neural_net

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Regression - homework

In [9]:
dropbox_path = '/Users/nd/projects/pml-neuralnet-hw/PersonalizedML/'
filename = 'parkinsons_supervised.csv'

In [3]:
# Set the hyperparameters
layer_sizes = [128,32]
batch_size = 25
learning_rate = .001
dropout_prob = 1.0
weight_penalty = 0.0
clip_gradients = False

In [6]:
reload(neural_net)
neural_net.reload_files()

In [10]:
net = neural_net.NeuralNetwork(dropbox_path + filename, 'homework', 
                               layer_sizes, batch_size, learning_rate, dropout_prob, 
                               weight_penalty, clip_gradients, model_type='regression')

Original data length was 5875
After dropping rows with nan in any label column, length is 5875
3581 rows in training data
1167 rows in validation data
1127 rows in testing data

Performing regression.
Input dimensions (number of features): 20
Number of classes/outputs: 2

Building computation graph...
Okay, making a neural net with the following structure:
[('20x128', '128'), ('128x32', '32'), ('32x2', '2')]


In [ ]:
net.train(num_steps=1000, output_every_nth=100)

In [ ]:
net.plot_training_progress()

### debugging neural network using session.run

In [ ]:
X, Y = net.data_loader.get_train_batch(10)
feed_dict = {net.tf_X: X,
             net.tf_Y: Y,
             net.tf_dropout_prob: 1.0}
print "X", np.shape(X), "Y", np.shape(Y)

In [ ]:
tf_X, tf_Y, logits, sq_err, rmse, loss = net.session.run([net.tf_X, 
                                                          net.tf_Y, 
                                                          net.logits,
                                                          n], feed_dict=feed_dict)
print "X", np.shape(tf_X), "Y", np.shape(tf_Y), "logits", np.shape(logits)

In [ ]:
# To see the tensorflow model you've built in tensorboard, run the following
journalist = tf.summary.FileWriter(net.checkpoint_dir, graph=net.session.graph)
journalist.flush()

# Now enter the following command at the command line
# > tensorboard --logdir 'saved_models'
# (you may need to replace 'saved_models' with the path to the actual 
# net.checkpoint_dir you are using)